<a href="https://colab.research.google.com/github/weathon/citrushack/blob/main/anti_pineapple_on_pizza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
!mv kaggle.json /root/.kaggle/kaggle.json
!kaggle datasets download -d sriramr/pizza-type-hawaiian-peppperoni

 99% 217M/219M [00:03<00:00, 49.9MB/s]
100% 219M/219M [00:03<00:00, 62.8MB/s]


In [ ]:
!unzip pizza-type-hawaiian-peppperoni.zip

In [ ]:
# We used the dataset from https://github.com/toppev/is-pineapple-pizza but code our own project

In [202]:
image_w , image_h = 128,128

model = tf.keras.models.Sequential([
  # RBG, because pineapple is yellow
  tf.keras.layers.Conv2D(6, (2,2), activation='relu', input_shape=(image_h,image_w,3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(8, (2,2), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Conv2D(16, (2,2), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Conv2D(16, (2,2), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512,activation="relu"),
  tf.keras.layers.Dense(258,activation="relu"),
  tf.keras.layers.Dense(128,activation="relu"),
  tf.keras.layers.Dropout(0.5), #toohigh?kunkesou
  tf.keras.layers.Dense(64,activation="relu"),
  tf.keras.layers.Dense(10,activation="relu"),
  tf.keras.layers.Dense(1,activation="sigmoid"),
])
model.summary()
# print(model.output_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss='binary_crossentropy',metrics=["accuracy"])


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 127, 127, 6)       78        
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 63, 63, 6)        0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 62, 62, 8)         200       
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 31, 31, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_46 (Conv2D)          (None, 30, 30, 16)        528       
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 15, 15, 16)     

In [ ]:
!wget http://pizzagan.csail.mit.edu/pizzaGANdata.zip

!unzip pizzaGANdata.zip

In [ ]:
! mkdir not_pineapple
! mkdir pineapple

#  https://github.com/toppev/is-pineapple-pizza/blob/main/dataset/categorize.py
import os
from shutil import copyfile

images = sorted(os.listdir("pizzaGANdata/images"))

max_images = 5000
test_images = 50

pineapples = 0
not_pineapples = 0

with open("pizzaGANdata/imageLabels.txt") as f:
    index = 0
    for line in f:
        index += 1

        is_pineapple = line.endswith('1\n')
        target = 'not_pineapple/' + str(not_pineapples) + '.jpg'

        if is_pineapple:
            target = 'pineapple/' + str(pineapples) + '.jpg'
            if pineapples >= max_images:
                continue
            pineapples += 1
        else:
            if not_pineapples >= max_images:
                continue
            not_pineapples += 1
        if len(images) > index:
            copyfile('pizzaGANdata/images/' + images[index - 1], target)

print('Pineapples: ' + str(pineapples))
print('Not pineapples: ' + str(not_pineapples))

Pineapples: 83
Not pineapples: 5000


In [ ]:
!cat pizzaGANdata/imageLabels.txt

In [163]:
X = []
Y = []

In [160]:
pineapples

83

In [159]:
len(X)

353

In [164]:
import random
not_pineapples_list = random.sample(os.listdir("not_pineapple"),pineapples*5)
pineapples_list = os.listdir("pineapple")

In [165]:
for i in not_pineapples_list:
  X.append(np.array(Image.open("not_pineapple/"+i).resize((128,128)).convert("RGB")))
  Y.append(0)

In [166]:
for i in pineapples_list:
  X.append(np.array(Image.open("pineapple/"+i).resize((128,128)).convert("RGB")))
  Y.append(1)

for i in os.listdir("hawaiian_pizza"):
  X.append(np.array(Image.open("hawaiian_pizza/"+i).resize((128,128)).convert("RGB")))
  Y.append(1)

In [167]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
model.fit(X, Y, epochs=40, shuffle=True)

In [210]:
model(np.array([np.array(Image.open("3.jpg").resize((128,128)).convert("RGB"))]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8214454]], dtype=float32)>

In [ ]:
Y[5]

0

In [ ]:
# Testing Area

In [ ]:
import base64

In [ ]:
code = input("Please Paste The Base64 Code Here: \n")
import base64
#  https://jdhao.github.io/2020/03/17/base64_opencv_pil_image_conversion/ part of the code (about 2~3 lines)
from io import BytesIO

im_bytes = base64.b64decode(code)   # im_bytes is a binary image
im_file = BytesIO(im_bytes)  # convert image to file-like object
img = Image.open(im_file).resize((128,128)).convert("RGB")   # img is now PIL Image object

# image = base64.decode(code)
ans = model.predict(np.array([np.array(img)])) #ttyunwangjile[0]
print("This Pizza Is %s likely t be evil!!!" % ans * 100)

Please Paste The Base64 Code Here: 
...
This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t be evil!!!This Pizza Is [[1.3433768e-06]] likely t b

In [ ]:
im_file